# API Face Samples

## Objective

This script uses a test image to tag faces by matching the detected face against a list of enrolled persons or a dynamic person group.

In [ ]:
import os
from shared_functions import identify_faces, delete_dynamic_person_group, delete_person
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

def plot_faces_on_image(image_path, identified_faces):
    # Load the original image
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    
    # Load a font
    try:
        font = ImageFont.truetype("arial.ttf", 20)
    except IOError:
        font = ImageFont.load_default()

    # Iterate over the identified faces and draw bounding boxes and names
    for face in identified_faces:
        bbox = face['bbox']
        name = face['personName'] if face['personName'] else "Unknown"
        
        # Define the bounding box coordinates
        left = bbox['left']
        top = bbox['top']
        right = left + bbox['width']
        bottom = top + bbox['height']
        
        # Draw the bounding box
        draw.rectangle([(left, top), (right, bottom)], outline="green", width=2)
        
        # Calculate text size using textbbox
        text_bbox = draw.textbbox((0, 0), name, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        
        # Draw a rectangle behind the text
        draw.rectangle([(left, max(top - text_height - 4, 0)), (left + text_width + 4, top)], fill="grey")
        
        # Draw the person's name on top of the bounding box
        draw.text((left + 2, top - text_height - 2), name, fill="white", font=font)
    
    # Display the image with bounding boxes
    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.axis('off')  # Hide axes
    plt.show()

# Replace with your Azure Face API subscription key and endpoint
FACE_KEY = os.environ["FACE_API_KEY"]
FACE_ENDPOINT = os.environ["FACE_ENDPOINT_URL"]

test_image_path = "../data/family.jpg" # Path to the test image
dynamic_person_group_id = "dynamic_person_group_id" # Created from enroll_single_face_per_person.py or enroll_multiple_faces_per_person.py
persond_ids = "person_ids" # Created from enroll_single_face_per_person.py or enroll_multiple_faces_per_person.py

# Tag faces in the test image using the person IDs
tagged_faces_by_ids = identify_faces(FACE_KEY, FACE_ENDPOINT, test_image_path, person_ids=persond_ids, injection_header="sample=tag_faces")

# Tag faces in the test image using the dynamic person group ID
tagged_faces_by_group = identify_faces(FACE_KEY, FACE_ENDPOINT, test_image_path, dynamic_person_group_id=dynamic_person_group_id, injection_header="sample=tag_faces")

# Plot the tagged faces on the test image
plot_faces_on_image(test_image_path, tagged_faces_by_group)

# Delete the dynamic person group (optional)
if delete_dynamic_person_group(FACE_KEY, FACE_ENDPOINT, dynamic_person_group_id, "sample=tag_faces"):
    print(f"Deleted dynamic person group with ID: {dynamic_person_group_id}")

# Delete the persons (optional)
for person_id in persond_ids:
    if delete_person(FACE_KEY, FACE_ENDPOINT, person_id, "sample=tag_faces"):
        print(f"Deleted person with ID: {person_id}")

print("Done")